## GoogleDriveをマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Train、Testフォルダの下にあるフォルダ名で分類する

### 調整箇所 パスなどを変更してください。
LEARNING_RATEは学習がうまくいかないときに、1e-3,1e-4,1e-5などを試してください。

In [0]:
###############ここを調整#########################
LEARNING_RATE = 1e-3
TRAIN_DIR = '/content/drive/My Drive/Colab Notebooks/Train/'
TEST_DIR = '/content/drive/My Drive/Colab Notebooks/Test/'
###############################################

### ファイル読み込み部

In [2]:
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib import ticker
import numpy as np

import tensorflow as tf

from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD,Adam
from keras.models import Model, load_model, Sequential, model_from_json
from keras import backend as K

IMAGE_SIZE = 256
CHANNEL_SIZE = 3
MODEL_NAME = 'VGG19' 
BATCH_SIZE = 8
EPOCH_NUM = 30

%matplotlib inline

Using TensorFlow backend.


In [0]:
import os, sys, cv2, random
import os.path
import numpy as np
import zipfile

In [0]:
ZipFileList = []
DirectoryList = []
FileList = []
ClassCount = 0
ClassName = []

In [0]:
def applyLabels(dic,className):   
    tempArray = np.zeros(len(dic))
    tempArray[dic[className]] = 1
    return tempArray
    
def read(name,image_size): 
    #print(name)
    size = (image_size, image_size)
    temp = np.array(cv2.resize(cv2.imread(name, 1), size))
    temp.astype(np.float32)
    return (temp[:,:,:] / 255.0).astype(np.float32)

def ls(dirname):
    return [dirname + i for i in os.listdir(dirname)]

def readImage(filePath, image_size):
    #print("FileName" + filePath)
    images = np.array([read(filePath,image_size)])
    return images

In [0]:
DirectoryList = [TRAIN_DIR + i for i in os.listdir(TRAIN_DIR)]

In [0]:
ClassCount = len(DirectoryList)
for j in DirectoryList:
    tempAry =  j.split("/")
    className = tempAry[len(tempAry)-1]
    ClassName.append(className)
    tempFileList = [i for i in os.listdir(j)]
    for f in tempFileList:
        fileInfo = {'filePath':TRAIN_DIR + className + '/' + f, 
                    'fileName':f, 
                    'className':className, 
                    'zipFilePath':j
                   }
        FileList.append(fileInfo)

In [0]:
train_images = np.array([read(i["filePath"],IMAGE_SIZE) for i in FileList])
labelsTemp = np.array([i["className"] for i in FileList])

dicLabel = {}
j = 0
for i in ClassName:
    dicLabel[i] = j
    j+=1

train_labels = np.array([applyLabels(dicLabel,i) for i in labelsTemp])

In [0]:
ZipFileList = []
DirectoryList = []
FileList = []
#ClassCount = 0
#ClassName = []

In [0]:
DirectoryList = [TEST_DIR + i for i in os.listdir(TEST_DIR)]

In [0]:
#ClassCount = len(DirectoryList)
for j in DirectoryList:
    tempAry =  j.split("/")
    className = tempAry[len(tempAry)-1]
    #ClassName.append(className)
    tempFileList = [i for i in os.listdir(j)]
    for f in tempFileList:
        fileInfo = {'filePath':TEST_DIR + className + '/' + f, 
                    'fileName':f, 
                    'className':className, 
                    'zipFilePath':j
                   }
        FileList.append(fileInfo)

In [0]:
test_images = np.array([read(i["filePath"],IMAGE_SIZE) for i in FileList])
labelsTemp = np.array([i["className"] for i in FileList])

dicLabel = {}
j = 0
for i in ClassName:
    dicLabel[i] = j
    j+=1

test_labels = np.array([applyLabels(dicLabel,i) for i in labelsTemp])

In [43]:
print(ClassName)

['car', 'animal']


In [13]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(117, 256, 256, 3)
(117, 2)
(25, 256, 256, 3)
(25, 2)


### AIの設定部

In [14]:
base_model = VGG19(include_top=False, weights='imagenet', input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNEL_SIZE)))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
# 全結合層の構築
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, name='fc1'))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(ClassCount, name='fc2'))
top_model.add(Activation('sigmoid', name='out'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
# 全結合層を削除したモデルと上で自前で構築した全結合層を結合
model = Model(input=base_model.input, output=top_model(base_model.output))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  """Entry point for launching an IPython kernel.


In [0]:
# frozen 処理
layer_num = len(base_model.layers)
for layer in base_model.layers[:int(layer_num * 0.9)]:
        layer.trainable = False

In [0]:
model.compile(loss='binary_crossentropy',
                       optimizer=SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True),
                       metrics=['accuracy'])

### 学習

In [0]:
history = model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCH_NUM)

### テスト

In [45]:
score = model.evaluate(test_images, test_labels)
print('loss=', score[0])
print('accuracy=', score[1])

25/25 [==============================] - 22s 895ms/step
loss= 0.0005311223794706166
accuracy= 1.0


In [46]:
#img_path = '/content/drive/My Drive/Colab Notebooks/Test/car/IMG_1073.jpg'
img_path = '/content/drive/My Drive/Colab Notebooks/Test/animal/IMG_1141.jpg'
x = readImage(img_path, IMAGE_SIZE)
predicted = model.predict(x)[0]
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
print(np.sum(predicted))
for i, prob in enumerate(predicted):
    if prob > 0.7 :
        print('%15s is score %2.2f %%' % (ClassName[i],predicted[i]*100))
    else :
        print('%15s is score %2.2f %%' % (ClassName[i],predicted[i]*100))

1.0000603
            car is score 0.01 %
         animal is score 99.99 %
